# 트리 기반 머신 러닝 모델
- 의사결정 트리 : 단순 모델과 부류 가중값 튜닝 모델
- 배깅(부트스트랩 종합)
- 랜덤 포레스트 - 단순 랜덤 포레스트와 그리드 검색을 적용한 초매개변수 튜닝
- 부스팅(에이다 부스트, 경사법 부스트, 극단 경사법 부스트 -XG 부스트)
- 앙상블들의 앙상블(비균질 및 균질 모델)

## 결정 트리 분류기 소개
- 해석하기 쉽다
- 분류, 회귀 모두 적용 가능

## 결정 트리에 사용되는 용어들
### 엔트로피 
- 정보이론에서 가져온 개념
- 데이터 불순도에 관한 척도
- 표본 전체가 완전히 균질 -> 엔트로피 0
- 표본 전체가 완전히 균등하게 분할 -> 엔트로피 1
- 엔트로피는 작을수록 분류를 더 잘할 수 있으므로 작을수록 바람직하다

<h3> 엔트로피 = $ -p_{1}*log_{2}p_{1} -   ...   - p_{n}*log_{2}p_{n} $</h3>
n = 부류의 개수


### 정보 이득
- 엔트로피의 감소 기댓값
- 단계마다 최대 정보 이득값을 갖는 변수가 선택 => 엔트로피의 감소 기댓값이 가장 큰 변수가 선택

<h3>$ 정보 이득 = 부모 노드의 엔트로피 - sum(가중값 * 자식 노드의 엔트로피) $</h3>

### 지니
- 잘못된 분류를 측정하는 도구
- 다부류 분류기에 적용
- 엔트로피와 거의 동일. 하지만, 빠른 계산이 가능

<h3>$지니 = 1 - \sum_{i=1}^{} p_{i}^2$</h3>
i = 부류의 개수

## 기본 원리로 본 결정 트리의 작동원리(ONE NOTE 참조)

In [1]:
# 모듈 가져오기
import pandas as pd
import numpy as np

In [2]:
# 데이터 불러오기
tennis = pd.read_csv('./Data/tennis.csv')

In [3]:
# 데이터 확인하기
tennis

,일,전망,기온,습도,바람,테니스 유무
0,D1,맑음,높음,높음,약함,아니요
1,D2,맑음,높음,높음,강함,아니요
2,D3,흐림,높음,높음,약함,예
3,D4,비,보통,높음,약함,예
4,D5,비,낮음,보통,약함,예
5,D6,비,낮음,보통,강함,아니요
6,D7,흐림,낮음,보통,강함,예
7,D8,맑음,보통,높음,약함,아니요
8,D9,맑음,낮음,보통,약함,예
9,D10,비,보통,보통,약함,예


In [29]:
# 종속변수 => 테니스 유무
# crosstab을 이용해서 데이터프레임을 재구성하자
pd.crosstab(tennis['습도'], tennis['테니스 유무'])

테니스 유무,아니요,예
습도,,
높음,4,3
보통,1,6


In [46]:
# 카이제곱 검정
from scipy.stats import chi2_contingency
cross_tbl = pd.crosstab(tennis['습도'], tennis['테니스 유무'])
chi_res = chi2_contingency(cross_tbl)
print('Chi2 Statistic: {}, p-value: {}, degree of freedom : {}'.format(  round(chi_res[0],4) , round(chi_res[1],4) , chi_res[2]))
print(chi_res[3])

Chi2 Statistic: 1.2444, p-value: 0.2646, degree of freedom : 1
[[2.5 4.5]
 [2.5 4.5]]


## 로지스틱 회귀와 결정 트리 비교
|로지스틱 회귀|결정 트리|
|:--|:--|
|- 종속 변수에 관한 독립 변수의 등식과 비슷|- 단순한 문장으로 규칙을 만들므로 설명이 용이|
|- 독립변수와 매개변수의 곱으로 모델을 정의|- 비매개변수 모델|
|- 예측을 위해 종속변수를 이항 분포나 베르누이 분포로 만듬|- 어떠한 가정도 하지 않음|
|- 모델의 곡선이 사전에 정의|- 모델의 모양이 사전에 정해지지 않는다<br>- 데이터에 관해 최적 분류에 적합|
|- 독립변수가 연속이고 선형이 잘 유지될 때 효과적|- 모든 변수가 범주 변수일 때 효과적|
|- 변수 간의 복잡한 상호작용은 찾아내기 힘듬|- 변수간 비선형 관계가 트리의 성능에 영향 끼치지 않는다|
|- 이상값과 결측값이 전체적인 성능을 떨어뜨림|- 결정 트리는 이상값과 결측값을 매우 잘 처리|

## 이상적인 영역으로 가는 개선책
- 선형 회귀의 경우 높은 편향 성분을 가질 수 있음 => 선형 스플라인으로 해결
- 의사결정 트리는 높은 분산문제 => 앙상블을 수행함으로써 해결

## HR 퇴직률 데이터 예제

In [135]:
# 모듈 가져오기
import pandas as pd
hrattr_data = pd.read_csv('./Data/HR_Employee_Attrition.csv')

In [136]:
# 데이터 확인하기
hrattr_data

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [137]:
# 데이터 타입 확인하기
hrattr_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [138]:
# 데이터의 결측치는 없는 상황
# attrition 컬럼은 Yes, No를 1,0 으로 변환
hrattr_data['Attrition_ind'] = 0
hrattr_data.loc[hrattr_data['Attrition'] == 'Yes', 'Attrition_ind'] = 1

In [139]:
# 데이터 변환 확인
hrattr_data['Attrition_ind']

0       1
1       0
2       1
3       0
4       0
       ..
1465    0
1466    0
1467    0
1468    0
1469    0
Name: Attrition_ind, Length: 1470, dtype: int64

In [140]:
# object 변수와 int 변수를 분리해서 컬럼명을 지정하자
continuous_columns = list()
discrete_columns = list()
dtypes = list(hrattr_data.dtypes)
for dtype in enumerate(dtypes):
    if dtype[1] == 'object':
        discrete_columns.append(hrattr_data.columns[dtype[0]])
    elif dtype[1] == 'int64':
        continuous_columns.append(hrattr_data.columns[dtype[0]])
        
# 알파벳 순으로 정렬
discrete_columns = sorted(discrete_columns)
continuous_columns = sorted(continuous_columns)

In [141]:
# 변수확인
print(len(discrete_columns), discrete_columns)
print(len(continuous_columns), continuous_columns)

9 ['Attrition', 'BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime']
27 ['Age', 'Attrition_ind', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']


In [142]:
# 연속형변수 데이터 프레임 만들기
hrattr_continous = hrattr_data[continuous_columns]

In [143]:
# 범주형변수 더미변수 만들기
dummies_list = list()
for col in discrete_columns:
    dummies_list.append(pd.get_dummies(hrattr_data[col], prefix = col))   

In [144]:
# 더미변수를 이용해서 데이터 프레임 만들기
hrattr_discrete = pd.concat(dummies_list, axis = 1)

In [149]:
# 새로운 통합 데이터 프레임 만들기(범주형 + 연속형)
hrattr_data_new = pd.concat([hrattr_discrete, hrattr_continous], axis = 1)

In [150]:
# 데이터 프레임 확인하기
hrattr_data_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 58 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Attrition_No                       1470 non-null   uint8
 1   Attrition_Yes                      1470 non-null   uint8
 2   BusinessTravel_Non-Travel          1470 non-null   uint8
 3   BusinessTravel_Travel_Frequently   1470 non-null   uint8
 4   BusinessTravel_Travel_Rarely       1470 non-null   uint8
 5   Department_Human Resources         1470 non-null   uint8
 6   Department_Research & Development  1470 non-null   uint8
 7   Department_Sales                   1470 non-null   uint8
 8   EducationField_Human Resources     1470 non-null   uint8
 9   EducationField_Life Sciences       1470 non-null   uint8
 10  EducationField_Marketing           1470 non-null   uint8
 11  EducationField_Medical             1470 non-null   uint8
 12  EducationField_Other

In [151]:
# 필요없는 컬럼 제거하기
remove_cols = ['Attrition_No', 'Attrition_Yes', 'EmployeeCount', 'EmployeeNumber', 'Over18_Y', 'StandardHours']
hrattr_data_new.drop(remove_cols, axis = 1, inplace = True)

In [152]:
# 데이터 프레임 재확인
hrattr_data_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 52 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   BusinessTravel_Non-Travel          1470 non-null   uint8
 1   BusinessTravel_Travel_Frequently   1470 non-null   uint8
 2   BusinessTravel_Travel_Rarely       1470 non-null   uint8
 3   Department_Human Resources         1470 non-null   uint8
 4   Department_Research & Development  1470 non-null   uint8
 5   Department_Sales                   1470 non-null   uint8
 6   EducationField_Human Resources     1470 non-null   uint8
 7   EducationField_Life Sciences       1470 non-null   uint8
 8   EducationField_Marketing           1470 non-null   uint8
 9   EducationField_Medical             1470 non-null   uint8
 10  EducationField_Other               1470 non-null   uint8
 11  EducationField_Technical Degree    1470 non-null   uint8
 12  Gender_Female       

### (참고) 범주형 변수에 관해 추가 더미 변수를 없애지 않은 이유는 다중공선성이 별다른 문제를 일으키기 않기 때문

In [156]:
# train, test 데이터 분할
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(hrattr_data_new.drop('Attrition_ind', axis = 1), hrattr_data_new['Attrition_ind'], train_size = 0.7, random_state = 42)

In [157]:
# 데이터 분할상태 확인
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1029, 51), (441, 51), (1029,), (441,))

## 의사결정 트리(DT) 분류기
- 트리의 최대 깊이 : 5
- 분할을 위한 최소 관측값 : 2
- 각 노드에서 필요한 최소 관측값 : 1
- 불순도 축증 : 지니

In [158]:
# 결정 트리 분류기
from sklearn.tree import DecisionTreeClassifier

In [160]:
# 모델 생성 및 학습
dt_fit = DecisionTreeClassifier(criterion = 'gini', max_depth = 5, min_samples_split = 2, min_samples_leaf = 1, random_state = 42)
dt_fit.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=5, random_state=42)

In [162]:
# 학습 예측값
y_pred = dt_fit.predict(x_train)
# 교차검증표 작성
print(pd.crosstab(y_train, y_pred, rownames = ['Actual'], colnames = ['Predicted']))
# 예측 정확도
from sklearn.metrics import accuracy_score, classification_report
print(round(accuracy_score(y_train,y_pred),4), '\n', classification_report(y_train, y_pred))

In [168]:
# 테스트 예측값
y_pred_test = dt_fit.predict(x_test)
# 교차검증표 작성
print(pd.crosstab(y_test, y_pred_test, rownames = ['Actual'], colnames = ['Predicted']))
# 예측 정확도
from sklearn.metrics import accuracy_score, classification_report
print(round(accuracy_score(y_test,y_pred_test),4), '\n', classification_report(y_test, y_pred_test))

Predicted    0   1
Actual            
0          361  19
1           49  12
0.8458 
               precision    recall  f1-score   support

           0       0.88      0.95      0.91       380
           1       0.39      0.20      0.26        61

    accuracy                           0.85       441
   macro avg       0.63      0.57      0.59       441
weighted avg       0.81      0.85      0.82       441



## 트리 분류기의 가중값 튜닝

In [217]:
import numpy as np
dummyarray = np.empty((6,10))
dt_wttune = pd.DataFrame(dummyarray)

In [218]:
dt_wttune.columns = ['zero_wght', 'one_wght', 'tr_accuracy', 'tst_accuracy', 'prec_zero', 'prec_one', 'prec_ovll', 'recl_zero', 'recl_one', 'recl_ovll']

In [219]:
# Attrition_ind의 1값에 가중을 주려고 하는게 목적
zero_clwghts = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5]
for i in range(len(zero_clwghts)):
    clwght = {
        0: zero_clwghts[i],
        1: 1.0 - zero_clwghts[i]}
    df_fit = DecisionTreeClassifier(
        criterion = 'gini',
        max_depth = 5,
        min_samples_split = 2,
        min_samples_leaf = 1,
        random_state = 42,
        class_weight = clwght
    )
    df_fit.fit(x_train, y_train)
    y_pred = df_fit.predict(x_train)
    y_pred_test = df_fit.predict(x_test)
    dt_fit.fit(x_train, y_train)
    dt_wttune.loc[i, 'zero_wght'] = clwght[0]
    dt_wttune.loc[i, 'one_wght'] = clwght[1]
    dt_wttune.loc[i, 'tr_accuracy'] = round(accuracy_score(y_train, y_pred),4)
    dt_wttune.loc[i, 'tst_accuracy'] = round(accuracy_score(y_test, y_pred_test),4)
    clf_sp = classification_report(y_test, y_pred_test).split()
    dt_wttune.loc[i, 'prec_zero'] = float(clf_sp[5])
    dt_wttune.loc[i, 'prec_one'] = float(clf_sp[10])
    dt_wttune.loc[i, 'prec_ovll'] = float(clf_sp[19])
    dt_wttune.loc[i, 'recl_zero'] = float(clf_sp[6])
    dt_wttune.loc[i, 'recl_one'] = float(clf_sp[11])                                         
    dt_wttune.loc[i, 'recl_ovll'] = float(clf_sp[20])
    print("\nClass Weights", clwght, "Train Accuracy:", round(accuracy_score(y_train, y_pred),3) , "Test Accuracy:" ,round(accuracy_score(y_test, y_pred_test), 3))
    print("Test Confusion Matrix \n")
    print(pd.crosstab(y_test, y_pred_test, rownames = ['Actual'], colnames = ['Predicted']))


Class Weights {0: 0.01, 1: 0.99} Train Accuracy: 0.342 Test Accuracy: 0.272
Test Confusion Matrix 

Predicted   0    1
Actual            
0          65  315
1           6   55

Class Weights {0: 0.1, 1: 0.9} Train Accuracy: 0.806 Test Accuracy: 0.732
Test Confusion Matrix 

Predicted    0   1
Actual            
0          282  98
1           20  41

Class Weights {0: 0.2, 1: 0.8} Train Accuracy: 0.871 Test Accuracy: 0.83
Test Confusion Matrix 

Predicted    0   1
Actual            
0          341  39
1           36  25

Class Weights {0: 0.3, 1: 0.7} Train Accuracy: 0.881 Test Accuracy: 0.837
Test Confusion Matrix 

Predicted    0   1
Actual            
0          345  35
1           37  24

Class Weights {0: 0.4, 1: 0.6} Train Accuracy: 0.894 Test Accuracy: 0.832
Test Confusion Matrix 

Predicted    0   1
Actual            
0          346  34
1           40  21

Class Weights {0: 0.5, 1: 0.5} Train Accuracy: 0.896 Test Accuracy: 0.846
Test Confusion Matrix 

Predicted    0   1
Actual

In [221]:
dt_wttune

,zero_wght,one_wght,tr_accuracy,tst_accuracy,prec_zero,prec_one,prec_ovll,recl_zero,recl_one,recl_ovll
0,0.01,0.99,0.3421,0.2721,0.92,0.15,0.53,0.17,0.90,0.54
1,0.10,0.90,0.8056,0.7324,0.93,0.29,0.61,0.74,0.67,0.71
2,0.20,0.80,0.8707,0.8299,0.90,0.39,0.65,0.90,0.41,0.65
3,0.30,0.70,0.8814,0.8367,0.90,0.41,0.65,0.91,0.39,0.65
4,0.40,0.60,0.8941,0.8322,0.90,0.38,0.64,0.91,0.34,0.63
5,0.50,0.50,0.8960,0.8458,0.88,0.39,0.63,0.95,0.20,0.57
